In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv('../dataset/preprocessed_data.csv')

In [4]:
data.head()

,user_id,recipe_id,date,rating,review,minutes,contributor_id,submitted,tags,n_steps,...,sugar,sodium,protein,saturated_fat,carbohydrates,food_types,negative,neutral,positive,compound
0,492,20636,2002-12-01,4,this worked very well and is easy. i used not ...,20,56824,2002-10-27,"30-minutes-or-less, time-to-make, course, main...",5,...,39.0,5.0,4.0,11.0,5.0,Healthy,0.0,0.598,0.402,0.8553
1,8204,64566,2005-09-02,4,very good,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.238,0.762,0.4927
2,28657,64566,2005-12-22,5,better than the real,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.508,0.492,0.4404
3,36365,64566,2006-09-26,5,absolutely awesome i was speechless when i tri...,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.883,0.117,0.6590
4,20197,64566,2007-03-09,5,these taste absolutely wonderful my son in law...,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.675,0.325,0.8908


In [5]:
data = data[['user_id', 'minutes', 'n_steps', 'ingredients', 'n_ingredients', 'calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates', 'food_types', 'recipe_id']]

In [6]:
data.head()

,user_id,minutes,n_steps,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates,food_types,recipe_id
0,492,20,5,"devils food cake mix, vegetable oil, eggs, ree...",4,132.3,11.0,39.0,5.0,4.0,11.0,5.0,Healthy,20636
1,8204,40,10,"chicken tenders, flour, garlic powder, salt, g...",12,316.0,4.0,40.0,37.0,78.0,4.0,10.0,Non-veg,64566
2,28657,40,10,"chicken tenders, flour, garlic powder, salt, g...",12,316.0,4.0,40.0,37.0,78.0,4.0,10.0,Non-veg,64566
3,36365,40,10,"chicken tenders, flour, garlic powder, salt, g...",12,316.0,4.0,40.0,37.0,78.0,4.0,10.0,Non-veg,64566
4,20197,40,10,"chicken tenders, flour, garlic powder, salt, g...",12,316.0,4.0,40.0,37.0,78.0,4.0,10.0,Non-veg,64566


In [7]:
# print 3 rows consisting of ingredients and recipe_id
df = data[['ingredients', 'recipe_id']]
df.head(3)

,ingredients,recipe_id
0,"devils food cake mix, vegetable oil, eggs, ree...",20636
1,"chicken tenders, flour, garlic powder, salt, g...",64566
2,"chicken tenders, flour, garlic powder, salt, g...",64566


In [8]:
# find the count of duplicate rows
df.duplicated().sum()

763871

In [9]:
# drop duplicate rows
df.drop_duplicates(inplace=True)

/var/folders/3z/s0qdvzzd2kgc2kdxkfvxy9ym0000gn/T/ipykernel_15588/2547945980.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [10]:
df['ingredients'] = df['ingredients'].apply(lambda x: x.replace(', ', ' '))
df.head(3)

/var/folders/3z/s0qdvzzd2kgc2kdxkfvxy9ym0000gn/T/ipykernel_15588/31536878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ingredients'] = df['ingredients'].apply(lambda x: x.replace(', ', ' '))


,ingredients,recipe_id
0,devils food cake mix vegetable oil eggs reeses...,20636
1,chicken tenders flour garlic powder salt groun...,64566
8,hamburger breadcrumbs egg salt black pepper on...,92891


In [11]:
# Convert ingredients to numerical vectors
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['ingredients'])

In [12]:
ingredients = tokenizer.texts_to_sequences(df['ingredients'])
max_length = max([len(seq) for seq in ingredients])
max_length

78

In [13]:
# Pad the sequences to the max length
ingredients = tf.keras.preprocessing.sequence.pad_sequences(ingredients, maxlen=max_length)
ingredients.shape

(198249, 78)

In [14]:
# Convert recipe_id to numerical vectors
label_encoder = LabelEncoder()
recipe_ids = label_encoder.fit_transform(df['recipe_id'])

In [15]:
train_ingredients, test_ingredients, train_recipe_ids, test_recipe_ids = train_test_split(ingredients, recipe_ids, test_size=0.2)

In [16]:
print("Train ingredients shape: ", train_ingredients.shape)
print("Train recipe_ids shape: ", train_recipe_ids.shape)
print("Test ingredients shape: ", test_ingredients.shape)
print("Test recipe_ids shape: ", test_recipe_ids.shape)

Train ingredients shape:  (158599, 78)
Train recipe_ids shape:  (158599,)
Test ingredients shape:  (39650, 78)
Test recipe_ids shape:  (39650,)


In [17]:
# Define the stacked de-noising autoencoder
input_layer = Input(shape=(max_length,))
# x = Dense(2048, activation='relu')(input_layer)
# x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(input_layer)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
encoded = Dense(16, activation='relu')(x)
x = Dense(32, activation='relu')(encoded)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
# x = Dense(1024, activation='relu')(x)
# x = Dense(2048, activation='relu')(x)
decoded = Dense(max_length, activation='linear')(x)

Metal device set to: Apple M1 Pro


In [18]:
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='mse')

In [19]:
# Train the stacked de-noising autoencoder
autoencoder.fit(train_ingredients, train_ingredients, epochs=1, batch_size=32, validation_data=(test_ingredients, test_ingredients))

2023-04-29 15:00:35.672074: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4957/4957 [==============================] - 53s 11ms/step - loss: 3836.0188 - val_loss: 2622.9792


In [20]:
# Fine-tune the autoencoder with a supervised learning model
autoencoder.layers

In [21]:
# Get the encoder layers from the autoencoder
encoder_layers = autoencoder.layers[:5]
encoder_layers

In [22]:
# Define the classifier on top of the encoder layers
input_layer = encoder_layers[0].input
x = encoder_layers[-1].output
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)

In [23]:
# Create a new model with the encoder layers and the classifier
model = Model(inputs=input_layer, outputs=output_layer)

In [24]:
# Compile the model with binary_crossentropy loss and accuracy metric
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_ingredients, train_recipe_ids, epochs=100, batch_size=32, validation_data=(test_ingredients, test_recipe_ids))

Epoch 1/100
1746/4957 [=========>....................] - ETA: 29s - loss: -136775736854218539008.0000 - accuracy: 0.0000e+00

In [6]:
df.head(3)

,ingredients,recipe_id
0,"devils food cake mix, vegetable oil, eggs, ree...",20636
1,"chicken tenders, flour, garlic powder, salt, g...",64566
8,"hamburger, breadcrumbs, egg, salt, black peppe...",92891


In [10]:
# take first 200 rows
df = df[:10]

In [15]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers.noise import GaussianNoise
from sklearn.metrics.pairwise import cosine_similarity

# Load the data
recipes = df['recipe_id'].unique()
ingredients = np.unique(df['ingredients'].str.split(', ').values.flatten())

# Convert the ingredients to a one-hot encoding
one_hot_ingredients = np.zeros((len(recipes), len(ingredients)))
for i in range(len(recipes)):
    recipe_ingredients = data[data['recipe_id'] == recipes[i]]['ingredients'].values[0].split(', ')
    for ingredient in recipe_ingredients:
        one_hot_ingredients[i, np.where(ingredients == ingredient)] = 1

# Create train and test datasets
train_size = int(0.8 * one_hot_ingredients.shape[0])
train_recipes = one_hot_ingredients[:train_size, :]
test_recipes = one_hot_ingredients[train_size:, :]

# Define the model architecture
input_dim = train_recipes.shape[1]
hidden_dim1 = 256
hidden_dim2 = 128

input_layer = Input(shape=(input_dim,))
noise_layer = GaussianNoise(0.1)(input_layer)
hidden_layer1 = Dense(hidden_dim1, activation='relu')(noise_layer)
hidden_layer2 = Dense(hidden_dim2, activation='relu')(hidden_layer1)
encoding_layer = Dense(hidden_dim2, activation='relu')(hidden_layer2)
hidden_layer3 = Dense(hidden_dim1, activation='relu')(encoding_layer)
output_layer = Dense(input_dim, activation='sigmoid')(hidden_layer3)

model = Model(input_layer, output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.fit(train_recipes, train_recipes, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stop])

Epoch 1/100
1/1 [==============================] - 0s 479ms/step - loss: 0.6964 - val_loss: 0.6913
Epoch 2/100
1/1 [==============================] - 0s 36ms/step - loss: 0.6906 - val_loss: 0.6890
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 0.6863 - val_loss: 0.6861
Epoch 4/100
1/1 [==============================] - 0s 39ms/step - loss: 0.6817 - val_loss: 0.6825
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 0.6763 - val_loss: 0.6781
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 0.6696 - val_loss: 0.6727
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 0.6615 - val_loss: 0.6661
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 0.6516 - val_loss: 0.6578
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 0.6395 - val_loss: 0.6474
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.6249 - val_loss: 0.6347
Epoch 11/100
1/1 [

In [16]:
# Extract the encoder part of the model
encoder = Model(input_layer, encoding_layer)

In [17]:
# Use the encoder to predict recipes based on ingredients
new_ingredients = ['devils food cake mix', 'vegetable oil', 'eggs', 'reeses pieces']
input_vec = np.zeros((1, len(ingredients)))

In [18]:
for i in range(len(ingredients)):
    if ingredients[i] in new_ingredients:
        input_vec[0, i] = 1
test_pred = encoder.predict(input_vec)

1/1 [==============================] - 0s 41ms/step


In [22]:
# Find the closest recipe to a given set of ingredients
def find_closest_recipe(_ingredients, recipes, encoder):
    # Encode the ingredients
    input_vec = np.zeros((1, len(ingredients)))
    for i in range(len(ingredients)):
        if ingredients[i] in _ingredients:
            input_vec[0, i] = 1

    print(input_vec)
    encoding = encoder.predict(input_vec)

    # Compute the distances between the encoding and the recipe encodings
    similarities = cosine_similarity(encoding, recipes)

    # Find the index of the closest recipe
    closest_recipe_idx = np.argmax(similarities)

    # Return the ID of the closest recipe
    return data[data['recipe_id'] == recipes[closest_recipe_idx]]['recipe_id'].values[0]

# Test the model by finding the closest recipe to the new ingredients
closest_recipe_id = find_closest_recipe(new_ingredients, encoder=encoder, recipes=test_recipes)
print('Closest recipe ID:', closest_recipe_id)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1/1 [==============================] - 0s 10ms/step


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 128 while Y.shape[1] == 10